<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Convolve Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> The Convolve() function applies a series representing a digital filter to a time series by convolving the two series. The digital filter can be of any type such as low-pass, band-pass, band-reject, high-pass, and so on. User can use digital filters to separate time series that have been combined and to restore time series that have become distorted.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    TDSeries,
    Convolve,
    remove_context
    )

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Convolve_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["xconvolve_complex_leftmulti", "hconvolve_complex_rightmulti",
                              "xconvolve_complex_left", "hconvolve_complex_right"]) 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data1 = DataFrame.from_table("xconvolve_complex_leftmulti")
data2 = DataFrame.from_table("hconvolve_complex_rightmulti")
data3 = DataFrame.from_table("xconvolve_complex_left")
data4 = DataFrame.from_table("hconvolve_complex_right")

In [ ]:
data1

In [ ]:
data2

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>3. Example 1</b>
<p style = 'font-size:16px;font-family:Arial'>Execute Convolve() function to convolve two series into new time series using "CONV_DFFT" algorithm.</p>

<p style = 'font-size:16px;font-family:Arial'>The two input time series "payload_content" are 'MULTIVAR_COMPLEX' and the resultant time series "payload_content" is 'MULTIVAR_COMPLEX'.</p>

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
data1_series_df = TDSeries(data=data1,
                               id="id",
                               row_index_style="sequence",
                               row_index="seq",
                               payload_field=["a_real", "a_imag", "b_real", "b_imag", "c_real", "c_imag"],
                               payload_content="MULTIVAR_COMPLEX")    

In [ ]:
data1_series_df

In [ ]:
data2_series_df = TDSeries(data=data2,
                               id="id",
                               row_index_style="sequence",
                               row_index="seq",
                               payload_field=["a_real", "a_imag", "b_real", "b_imag", "c_real", "c_imag"],
                               payload_content="MULTIVAR_COMPLEX")

<p style = 'font-size:16px;font-family:Arial'>Convolve the "data1_series_df" and "data2_series_df" series using the Convolve() function. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(Convolve)

<p style = 'font-size:16px;font-family:Arial'><b>Input Parameters:</b></p>
<li style = 'font-size:16px;font-family:Arial'><code>data1:</code> Specifies the time series to be filtered.</li>
<li style = 'font-size:16px;font-family:Arial'><code>data2:</code> Specifies the actual filter kernel.</li>
<li style = 'font-size:16px;font-family:Arial'><code>algorithm:</code> Specifies the options to use for convolving. The permitted values are: <b>CONV_SUMMATION, CONV_DFFT</b></li>
<li style = 'font-size:16px;font-family:Arial'><code>input_fmt_input_mode:</code> Specifies the input mode supported by the function. The permitted values are: <b>ONE2ONE, MANY2ONE, MATCH</b></li>

In [ ]:
uaf_out1 = Convolve(data1=data1_series_df,
                        data2=data2_series_df,
                        algorithm="CONV_DFFT",
                        input_fmt_input_mode="MATCH")
 
# Print the result DataFrame.
uaf_out1.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>4. Example 2</b>
<p style = 'font-size:16px;font-family:Arial'>Execute Convolve() function to convolve two series into new time series using "CONV_SUMMATION" algorithm.</p>

<p style = 'font-size:16px;font-family:Arial'>The two input time series "payload_content" are 'MULTIVAR_REAL' and the resultant time series "payload_content" is 'MULTIVAR_COMPLEX'.</p>

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
data3_series_df = TDSeries(data=data3,
                               id="id",
                               row_index_style="sequence",
                               row_index="seq",
                               payload_field=["a_real", "a_imag"],
                               payload_content="MULTIVAR_REAL")  

In [ ]:
data4_series_df = TDSeries(data=data4,
                               id="id",
                               row_index_style="sequence",
                               row_index="seq",
                               payload_field=["a_real", "a_imag"],
                               payload_content="MULTIVAR_REAL")

<p style = 'font-size:16px;font-family:Arial'>Convolve the "data3_series_df" and "data4_series_df" series using the Convolve() function.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Input Parameters:</b></p>
<li style = 'font-size:16px;font-family:Arial'><code>data1:</code> Specifies the time series to be filtered.</li>
<li style = 'font-size:16px;font-family:Arial'><code>data2:</code> Specifies the actual filter kernel.</li>
<li style = 'font-size:16px;font-family:Arial'><code>algorithm:</code> Specifies the options to use for convolving. The permitted values are: <b>CONV_SUMMATION, CONV_DFFT</b></li>
<li style = 'font-size:16px;font-family:Arial'><code>input_fmt_input_mode:</code> Specifies the input mode supported by the function. The permitted values are: <b>ONE2ONE, MANY2ONE, MATCH</b></li>


In [ ]:
uaf_out2 = Convolve(data1=data3_series_df,
                        data2=data4_series_df,
                        algorithm="CONV_SUMMATION",
                        input_fmt_input_mode="MATCH")

# Print the result DataFrame.
uaf_out2.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Convolve function reference: <a href = 'https://docs.teradata.com/search/all?query=Convolve&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>